In [1]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.notebook_imports import *
from pyforest import *
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from src.data.Dataset import create_3d_volumes_from_4d_files, create_4d_volumes_from_4d_files, create_2d_slices_from_4d_volume_files
from src.utils.utils_io import Console_and_file_logger, ensure_dir
Console_and_file_logger('create_4D_3D_2D_data_for_gcn_axial')

search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio


Using TensorFlow backend.
2020-07-06 11:00:29,088 INFO -------------------- Start --------------------
2020-07-06 11:00:29,102 INFO Working directory: /mnt/data/git/cardio.
2020-07-06 11:00:29,102 INFO Log file: ./logs/create_4D_3D_2D_data_for_gcn_axial.log
2020-07-06 11:00:29,102 INFO Log level for console: INFO


In [2]:
# select folder with the images and masks
# test with interativ filechooser
from ipyfilechooser import FileChooser
config_chooser = FileChooser(os.path.join(os.getcwd(),'data/raw'), '')
display(config_chooser)

FileChooser(path='/mnt/data/git/cardio/data/raw', filename='', show_hidden='False')

In [19]:
import re
def split_helper(string):
    # extract the patient id without the pre numbering
    return re.split('[0-9][0-9][0-9][0-9]-',os.path.basename(string))[1].split('_')[0]

raw_data_path = config_chooser.selected
# read the raw imported 4D files
logging.info('import path raw: {}'.format(raw_data_path))
raw_img_reg = os.path.join(raw_data_path, '*clean.nrrd')
raw_msk_reg = os.path.join(raw_data_path, '*mask.nrrd')
images = sorted(glob.glob(raw_img_reg, recursive=True))
masks = sorted(glob.glob(raw_msk_reg, recursive=True))
logging.info(len(images))
logging.info(len(masks))
# create a dataframe for images and masks
patients = [split_helper(f) for f in images]
df = pd.DataFrame({'patient':patients, 'x_path': images, 'y_path':masks})
df.head()

2020-06-09 10:39:17,903 INFO import path raw: /mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86/SAX/
2020-06-09 10:39:17,904 INFO 86
2020-06-09 10:39:17,905 INFO 86


,patient,x_path,y_path
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...


In [20]:
# Filter the image and mask volume by contoured time-steps
# create 4D data

import_path = raw_data_path + 'raw'
export_path = raw_data_path[:-1] + '_4D'
filter_threshold = 2
ensure_dir(export_path)

# load the original 4D files, also with all timesteps (incl not contoured time steps)
logging.info(len(images))
logging.info(len(masks))
# create a dataframe for images and masks
patients = [split_helper(f) for f in images]
df = pd.DataFrame({'patient':patients, 'x_path': images, 'y_path':masks})

assert(len(df['x_path']) == len(df['y_path']))
logging.info('create 4D volumes from: {} with: {} files'.format(import_path, len(df['x_path'])))

# slice all 4D volumes, copy metadata and save them
steps, shapes = zip(*[create_4d_volumes_from_4d_files(img_f, mask_f, export_path, slice_threshold=filter_threshold) for img_f, mask_f in zip(df['x_path'], df['y_path'])])

# overwrite the dataframe if all files are copied
images_filtered = sorted(glob.glob(os.path.join(export_path, '*img.nrrd')))
masks_filtered = sorted(glob.glob(os.path.join(export_path, '*msk.nrrd')))
logging.info('created {} filtered 4D volumes from {} raw 4D volumes'.format(len(images_filtered), len(images)))
if len(df['x_path']) != len(images_filtered):
    logging.error('not all 4D volumes are processed/copied')
    patients = [split_helper(f) for f in images_filtered]
    df = pd.DataFrame({'patient':patients, 'x_path': images_filtered, 'y_path':masks_filtered})
else:
    df['x_path'] = images_filtered
    df['y_path'] = masks_filtered

# remember filtered timesteps
df['timesteps'] = steps
df['img_shape'] = shapes

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '4d_dataset.csv', index=False)
# save df to the directory above
df.to_csv(os.path.join(os.path.dirname(export_path),os.path.basename(export_path)+ '_dataset.csv'), index=False)
print(df.shape)
df.head()

2020-06-09 10:39:22,699 INFO 86
2020-06-09 10:39:22,699 INFO 86
2020-06-09 10:39:22,702 INFO create 4D volumes from: /mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86/SAX/raw with: 86 files
2020-06-09 10:39:22,703 INFO process file: /mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86/SAX/0000-0HQQW4ZN_2007-05-23_volume_clean.nrrd
2020-06-09 10:39:22,725 INFO (25, 10, 256, 256)
2020-06-09 10:39:22,735 INFO filtered timesteps: [6, 10, 15, 21, 24]
2020-06-09 10:39:22,771 INFO process file: /mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86/SAX/0000-0PTV75MP_2005-06-27_volume_clean.nrrd
2020-06-09 10:39:22,793 INFO (14, 12, 256, 256)
2020-06-09 10:39:22,800 INFO filtered timesteps: [0, 3, 7, 11, 13]
2020-06-09 10:39:22,836 INFO process file: /mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86/SAX/0000-0RPELLU8_2007-02-13_volume_clean.nrrd
2020-06-09 10:39:22,885 INFO (25, 13, 288, 288)
2020-06-09 10:39:22,900 INFO filtered timesteps: [3, 6, 11, 19, 24]
2020-06-09 10:39:22,956 INFO proce

(86, 5)


,patient,x_path,y_path,timesteps,img_shape
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 10, 15, 21, 24]","(5, 10, 256, 256)"
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[0, 3, 7, 11, 13]","(5, 12, 256, 256)"
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[3, 6, 11, 19, 24]","(5, 13, 288, 288)"
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[4, 8, 14, 22, 23]","(5, 14, 288, 288)"
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 12, 18, 22, 24]","(5, 16, 320, 320)"


In [21]:
df.head()

,patient,x_path,y_path,timesteps,img_shape
0,0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 10, 15, 21, 24]","(5, 10, 256, 256)"
1,0PTV75MP,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[0, 3, 7, 11, 13]","(5, 12, 256, 256)"
2,0RPELLU8,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[3, 6, 11, 19, 24]","(5, 13, 288, 288)"
3,11YU3CUF,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[4, 8, 14, 22, 23]","(5, 14, 288, 288)"
4,13JLP3HN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,"[6, 12, 18, 22, 24]","(5, 16, 320, 320)"


In [22]:
# create 3D data
import_path = raw_data_path + '3D'
export_path = raw_data_path[:-1] + '_3D'
filter_threshold = 2

ensure_dir(export_path)

# load the original 4D files, also with all timesteps (incl not contoured time steps)
logging.info('raw images: {}'.format(len(images)))
logging.info('raw masks: {}'.format(len(masks)))
# create a dataframe for images and masks
patients = [split_helper(f) for f in images]
df = pd.DataFrame({'patient':patients, 'x_path': images, 'y_path':masks})

assert(len(images) == len(masks))
logging.info('create 3D volumes from: {} with: {} files'.format(import_path, len(images)))

# slice all 4D volumes, copy metadata and save them
steps, shapes = zip(*[create_3d_volumes_from_4d_files(img_f, mask_f, export_path, slice_treshhold=filter_threshold) for img_f, mask_f in zip(images, masks)])

# overwrite the dataframe if all files are copied
images_filtered = sorted(glob.glob(os.path.join(export_path, '*img.nrrd')))
masks_filtered = sorted(glob.glob(os.path.join(export_path, '*msk.nrrd')))
logging.info('created {} filtered 3D volumes from {} raw 4D volumes'.format(len(images_filtered), len(images)))
patients = [f.split('/')[-1].split('_')[0] for f in images_filtered]
df = pd.DataFrame({'patient': patients, 'x_path':images_filtered, 'y_path': masks_filtered})

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '3d_dataset.csv')

# save df to the directory above
df.to_csv(os.path.join(os.path.dirname(export_path), os.path.basename(export_path)+'_dataset.csv'), index=False)
print(df.shape)
df.head()

2020-06-09 10:39:32,739 INFO raw images: 86
2020-06-09 10:39:32,739 INFO raw masks: 86
2020-06-09 10:39:32,743 INFO create 3D volumes from: /mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86/SAX/3D with: 86 files
2020-06-09 10:39:32,743 INFO process file: /mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86/SAX/0000-0HQQW4ZN_2007-05-23_volume_clean.nrrd
2020-06-09 10:39:32,774 INFO (25, 10, 256, 256)
2020-06-09 10:39:32,783 INFO filtered timesteps: [6, 10, 15, 21, 24]
2020-06-09 10:39:32,821 INFO process file: /mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86/SAX/0000-0PTV75MP_2005-06-27_volume_clean.nrrd
2020-06-09 10:39:32,835 INFO (14, 12, 256, 256)
2020-06-09 10:39:32,843 INFO filtered timesteps: [0, 3, 7, 11, 13]
2020-06-09 10:39:32,875 INFO process file: /mnt/data/git/cardio/data/raw/gcn_05_2020_ax_sax_86/SAX/0000-0RPELLU8_2007-02-13_volume_clean.nrrd
2020-06-09 10:39:32,922 INFO (25, 13, 288, 288)
2020-06-09 10:39:32,937 INFO filtered timesteps: [3, 6, 11, 19, 24]
2020-06-09 1

(429, 3)


,patient,x_path,y_path
0,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
1,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
2,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
3,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...
4,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...,/mnt/data/git/cardio/data/raw/gcn_05_2020_ax_s...


In [12]:
# create 2D data from 4D raw volumes
import_path = raw_data_path + '2D'
export_path = raw_data_path[:-1] + '_2D'
filter_threshold = 2
only_masked = True

ensure_dir(export_path)

# load the original 4D files, also with all timesteps (incl not contoured time steps)
logging.info('raw images: {}'.format(len(images)))
logging.info('raw masks: {}'.format(len(masks)))
# create a dataframe for images and masks
patients = [split_helper(f) for f in images]
df = pd.DataFrame({'patient':patients, 'x_path': images, 'y_path':masks})

assert(len(images) == len(masks))
logging.info('create 2D slices from: {} with: {} files'.format(import_path, len(images)))

# slice all 4D volumes, copy metadata and save them
steps, shapes = zip(*[create_2d_slices_from_4d_volume_files(img_f, mask_f, export_path, only_masked, slice_threshold=filter_threshold) for img_f, mask_f in zip(images, masks)])

# overwrite the dataframe if all files are copied
images_filtered = sorted(glob.glob(os.path.join(export_path, '*img.nrrd')))
masks_filtered = sorted(glob.glob(os.path.join(export_path, '*msk.nrrd')))
logging.info('created {} filtered 2D volumes from {} raw 4D volumes'.format(len(images_filtered), len(images)))
patients = [f.split('/')[-1].split('_')[0] for f in images_filtered]
df = pd.DataFrame({'patient': patients, 'x_path':images_filtered, 'y_path': masks_filtered})

# save dataframe to disk for later processing
df.to_csv('reports/'+ import_path.replace('/','_') + '2d_dataset.csv', index=False)

# save df to the directory above
df.to_csv(os.path.join(os.path.dirname(export_path), os.path.basename(export_path)+ '_dataset.csv'), index=False)
print(df.shape)
df.head()

2020-06-08 16:04:39,666 INFO raw images: 92
2020-06-08 16:04:39,667 INFO raw masks: 92
2020-06-08 16:04:39,670 INFO create 2D slices from: /mnt/data/git/cardio/data/raw/gcn_ax_100/AX/2D with: 92 files
2020-06-08 16:04:39,670 INFO process file: /mnt/data/git/cardio/data/raw/gcn_ax_100/AX/0000-0HQQW4ZN_2007-05-23_volume_clean.nrrd
2020-06-08 16:04:39,748 INFO (25, 24, 256, 256)
2020-06-08 16:04:39,767 INFO filtered timesteps: [10, 24]
2020-06-08 16:04:39,883 INFO process file: /mnt/data/git/cardio/data/raw/gcn_ax_100/AX/0000-0PTV75MP_2005-06-27_volume_clean.nrrd
2020-06-08 16:04:39,912 INFO (16, 16, 256, 256)
2020-06-08 16:04:39,920 INFO filtered timesteps: [0, 7]
2020-06-08 16:04:39,994 INFO process file: /mnt/data/git/cardio/data/raw/gcn_ax_100/AX/0000-0RPELLU8_2007-02-13_volume_clean.nrrd
2020-06-08 16:04:40,062 INFO (25, 20, 288, 288)
2020-06-08 16:04:40,083 INFO filtered timesteps: [6, 24]
2020-06-08 16:04:40,191 INFO process file: /mnt/data/git/cardio/data/raw/gcn_ax_100/AX/0000-11

(4156, 3)


,patient,x_path,y_path
0,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...
1,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...
2,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...
3,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...
4,0000-0HQQW4ZN,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...,/mnt/data/git/cardio/data/raw/gcn_ax_100/AX_2D...
